In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)
tqdm.tqdm.pandas()
#import multiprocess as mp

In [2]:
%%time
main_data_df = pd.read_csv('./Prepped Data/main_data_coor_days_nonan_20211224.csv', dtype={1:'str', 9:'Int64', 10:'Int64'})
main_data_df['계약날짜'] = pd.to_datetime(main_data_df['계약날짜'])
print(main_data_df.shape)
main_data_df.head()

(5068169, 22)
Wall time: 13.4 s


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준
0,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,2253,12,12,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-02-22,4070
1,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,730,10,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-04-26,4133
2,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,1815,4,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-08-31,4260
3,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,3865,17,11,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2018-03-29,4470
4,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,3062,7,9,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2018-07-14,4577


In [3]:
main_data_df['지번주소'] = main_data_df['시군구'] + ' ' + main_data_df['번지']

In [4]:
main_data_df = main_data_df.sort_values(['시군구', '본번', '부번', '건축년도', '계약날짜']).reset_index(drop=True)
print(main_data_df.shape)
main_data_df.head()

(5068169, 23)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,지번주소
0,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,2253,12,12,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-02-22,4070,서울특별시 강남구 개포동 12
1,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,730,10,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-04-26,4133,서울특별시 강남구 개포동 12
2,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,1815,4,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-08-31,4260,서울특별시 강남구 개포동 12
3,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,3865,17,11,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2018-03-29,4470,서울특별시 강남구 개포동 12
4,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,3062,7,9,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2018-07-14,4577,서울특별시 강남구 개포동 12


In [5]:
main_data_df = main_data_df.rename(columns={'번지':'지번'})

In [6]:
official_price_df = pd.read_csv('./Prepped Data/official_price_20220124.csv')
print(official_price_df.shape)
official_price_df.head()

(30239919, 8)


,시군구명,법정동명,공시지가(원/㎡),기준년도,기준년월,본번,부번,대장구분명
0,강남구,개포동,170000,1990,1990-01-01,1,1,산
1,강남구,개포동,480000,1991,1991-01-01,1,1,산
2,강남구,개포동,1300000,1992,1992-01-01,1,1,산
3,강남구,개포동,1500000,1993,1993-01-01,1,1,산
4,강남구,개포동,1500000,1994,1994-01-01,1,1,산


In [7]:
main_data_df['구'] = main_data_df['시군구'].apply(lambda x: x.split(' ')[1])
main_data_df['동'] = main_data_df['시군구'].apply(lambda x: x.split(' ')[2])

In [8]:
main_data_df.head()

,시군구,지번,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,지번주소,구,동
0,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,2253,12,12,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-02-22,4070,서울특별시 강남구 개포동 12,강남구,개포동
1,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,730,10,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-04-26,4133,서울특별시 강남구 개포동 12,강남구,개포동
2,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,1815,4,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-08-31,4260,서울특별시 강남구 개포동 12,강남구,개포동
3,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,3865,17,11,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2018-03-29,4470,서울특별시 강남구 개포동 12,강남구,개포동
4,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,3062,7,9,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2018-07-14,4577,서울특별시 강남구 개포동 12,강남구,개포동


In [9]:
main_data_df['구'].value_counts()

송파구     423562
강서구     382628
강남구     346840
노원구     318121
서초구     257830
양천구     256701
강동구     256455
마포구     240839
은평구     240786
구로구     227283
영등포구    213418
관악구     191006
성북구     186953
동작구     183810
도봉구     172592
성동구     150291
동대문구    146140
광진구     142024
중랑구     139697
서대문구    139344
용산구     118866
강북구     114639
금천구      95779
중구       61645
종로구      60920
Name: 구, dtype: int64

In [10]:
main_data_df['동'].value_counts()

화곡동       142381
상계동       132349
봉천동       100126
목동         99346
신정동        95284
구로동        85050
신림동        80433
잠실동        76719
서초동        76474
창동         74469
중계동        65775
신월동        63438
역삼동        59557
방배동        58574
상도동        54765
대치동        54093
미아동        51831
등촌동        51379
천호동        49252
사당동        49123
월계동        48178
가락동        47217
방화동        47148
개포동        46954
문정동        46787
시흥동        45280
반포동        44415
공릉동        43935
자양동        43227
방이동        42993
개봉동        42112
마곡동        41188
암사동        40925
면목동        39140
정릉동        38684
장안동        38301
성내동        38018
쌍문동        37661
독산동        37501
수유동        37417
방학동        37376
구의동        36913
불광동        36072
도곡동        35694
응암동        35560
잠원동        35422
성산동        34397
가양동        33455
신길동        33235
신천동        33063
송파동        33040
신내동        32493
길동         32362
신당동        31517
삼성동        30595
신사동        30052
내발산동       30030
길음동        28671
염창동        283

In [11]:
official_price_df = official_price_df.rename(columns={'시군구명':'구', '법정동명':'동'})

In [12]:
official_price_df['구'].value_counts()

성북구     2132083
마포구     1703773
은평구     1601407
종로구     1561658
동대문구    1443027
관악구     1396246
강북구     1387389
서대문구    1383748
강서구     1345464
영등포구    1332100
동작구     1282451
중랑구     1222784
용산구     1204977
구로구     1193904
중구      1182530
성동구     1123912
강남구     1058020
서초구     1015824
송파구      986044
광진구      985389
강동구      897628
양천구      776094
도봉구      747302
노원구      638586
금천구      637579
Name: 구, dtype: int64

In [13]:
official_price_df['동'].value_counts()

신림동       761710
미아동       662325
봉천동       588590
화곡동       548206
신길동       523049
면목동       510283
수유동       498605
구로동       439015
신당동       376692
정릉동       369884
상도동       339880
장위동       334246
시흥동       311922
방배동       296160
신월동       295696
응암동       295087
사당동       288060
중곡동       284357
신정동       281679
상계동       278047
천호동       263652
독산동       263346
불광동       250972
자양동       249981
이문동       235410
전농동       234458
홍은동       227549
신사동       220797
쌍문동       216971
답십리동      212147
대림동       211676
개봉동       211345
목동        207401
길음동       202931
방화동       202843
창동        198172
홍제동       197317
망우동       192853
역삼동       192195
구의동       184910
서초동       184738
남가좌동      182997
석관동       179537
연희동       176863
논현동       173677
갈현동       173540
아현동       171579
종암동       171364
창신동       170420
행당동       169708
방학동       169183
흑석동       167513
중화동       165254
하월곡동      164522
북아현동      164423
제기동       164409
도봉동       162976
공덕동       161689
진관동       1616

In [14]:
main_data_df['계약날짜기준_건물연식'] = main_data_df['계약날짜'].dt.year - main_data_df['건축년도']

In [15]:
main_data_df['계약날짜기준_건물연식'].value_counts()

 0     301287
 4     208194
 1     207838
 2     198333
 6     186845
 5     185912
 3     179853
 12    179459
 13    177079
 8     177074
 10    176697
 9     173508
 14    173219
 11    172596
 7     171590
 15    171123
 16    167194
 17    157614
 18    150443
 19    132420
 20    122071
 21    111779
 22    103163
 23    102410
 24     97107
 25     94395
 26     90430
 27     87287
 28     83795
 29     76097
 30     67192
 31     62964
 32     59593
 33     53182
 34     40609
 35     32910
 36     23714
 37     20583
 38     17349
 39     12893
 40     11248
 41      9396
 42      8347
 43      6320
-1       6144
 44      4356
 45      3521
 46      2732
 47      2041
 48      1317
 49      1156
 50      1031
-2        744
-3        589
 51       372
 52       174
 53       131
-5         87
 54        85
-12        80
-4         79
 55        59
-7         49
-6         45
 56        44
 59        31
 57        30
-11        25
 58        25
 60        23
-10        21
-9    

In [16]:
main_data_df['계약날짜기준_건물연식'].min()

-12

In [17]:
min_df = main_data_df[main_data_df['계약날짜기준_건물연식'] == -12]
print(min_df.shape)
min_df.head()

(80, 26)


,시군구,지번,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,지번주소,구,동,계약날짜기준_건물연식
1981197,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,59.84,16000,0,15,2018,중계로8길 20,아파트,14000~16000,현대,현대,일반,임,207210.650109,460354.156828,2006-01-04,3,서울특별시 노원구 중계동 34-5,노원구,중계동,-12
1981198,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,59.84,15750,0,11,2018,중계로8길 20,아파트,14000~16000,현대,현대,일반,임,207210.650109,460354.156828,2006-01-20,19,서울특별시 노원구 중계동 34-5,노원구,중계동,-12
1981199,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,59.84,16200,0,13,2018,중계로8길 20,아파트,16000~18000,현대,현대,일반,임,207210.650109,460354.156828,2006-02-03,33,서울특별시 노원구 중계동 34-5,노원구,중계동,-12
1981200,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,59.84,15200,0,8,2018,중계로8길 20,아파트,14000~16000,현대,현대,일반,임,207210.650109,460354.156828,2006-02-16,46,서울특별시 노원구 중계동 34-5,노원구,중계동,-12
1981201,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,59.84,15400,0,12,2018,중계로8길 20,아파트,14000~16000,현대,현대,일반,임,207210.650109,460354.156828,2006-02-16,46,서울특별시 노원구 중계동 34-5,노원구,중계동,-12


In [18]:
min_df

,시군구,지번,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,지번주소,구,동,계약날짜기준_건물연식
1981197,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,59.84,16000,0,15,2018,중계로8길 20,아파트,14000~16000,현대,현대,일반,임,207210.650109,460354.156828,2006-01-04,3,서울특별시 노원구 중계동 34-5,노원구,중계동,-12
1981198,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,59.84,15750,0,11,2018,중계로8길 20,아파트,14000~16000,현대,현대,일반,임,207210.650109,460354.156828,2006-01-20,19,서울특별시 노원구 중계동 34-5,노원구,중계동,-12
1981199,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,59.84,16200,0,13,2018,중계로8길 20,아파트,16000~18000,현대,현대,일반,임,207210.650109,460354.156828,2006-02-03,33,서울특별시 노원구 중계동 34-5,노원구,중계동,-12
1981200,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,59.84,15200,0,8,2018,중계로8길 20,아파트,14000~16000,현대,현대,일반,임,207210.650109,460354.156828,2006-02-16,46,서울특별시 노원구 중계동 34-5,노원구,중계동,-12
1981201,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,59.84,15400,0,12,2018,중계로8길 20,아파트,14000~16000,현대,현대,일반,임,207210.650109,460354.156828,2006-02-16,46,서울특별시 노원구 중계동 34-5,노원구,중계동,-12
1981202,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,59.84,14000,0,2,2018,중계로8길 20,아파트,1억4천 이하,현대,현대,일반,임,207210.650109,460354.156828,2006-02-22,52,서울특별시 노원구 중계동 34-5,노원구,중계동,-12
1981203,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,84.98,28200,0,11,2018,중계로8길 20,아파트,28000~30000,현대,현대,일반,임,207210.650109,460354.156828,2006-03-06,64,서울특별시 노원구 중계동 34-5,노원구,중계동,-12
1981204,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,84.98,22900,0,2,2018,중계로8길 20,아파트,22000~24000,현대,현대,일반,임,207210.650109,460354.156828,2006-03-07,65,서울특별시 노원구 중계동 34-5,노원구,중계동,-12
1981205,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,84.98,29350,0,15,2018,중계로8길 20,아파트,28000~30000,현대,현대,일반,임,207210.650109,460354.156828,2006-03-08,66,서울특별시 노원구 중계동 34-5,노원구,중계동,-12
1981206,서울특별시 노원구 중계동,34-5,34,5,현대(6차),매매,59.84,16000,0,9,2018,중계로8길 20,아파트,14000~16000,현대,현대,일반,임,207210.650109,460354.156828,2006-03-08,66,서울특별시 노원구 중계동 34-5,노원구,중계동,-12


In [19]:
min_df['지번'].value_counts()

34-5    80
Name: 지번, dtype: int64

In [20]:
inv_df = main_data_df[(main_data_df['거래유형'] != '월세') & (main_data_df['월세(만원)'] != 0)]
print(inv_df.shape)
inv_df.head()

(17, 26)


,시군구,지번,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,지번주소,구,동,계약날짜기준_건물연식
189131,서울특별시 강남구 삼성동,141-10,141,10,삼성동 롯데캐슬 클라쎄,전세,21.87,24000,3,6,2019,테헤란로63길 9,오피스텔,22000~24000,삼성동 롯데캐슬 클라쎄,롯데캐슬,일반,대,204369.880744,444844.447272,2021-08-17,5707,서울특별시 강남구 삼성동 141-10,강남구,삼성동,2
804116,서울특별시 강서구 등촌동,631,631,0,등촌동두산위브센티움,전세,43.11,5000,60,3,2004,양천로 564,오피스텔,1억4천 이하,등촌동두산위브센티움,기타,일반,대,187682.690364,450578.892836,2021-06-21,5650,서울특별시 강서구 등촌동 631,강서구,등촌동,17
880707,서울특별시 강서구 마곡동,795-2,795,2,마곡아이파크,전세,36.98,2000,82,7,2016,마곡중앙8로 90,오피스텔,1억4천 이하,마곡아이파크,아이파크,일반,대,185590.593750,450933.823750,2021-07-27,5686,서울특별시 강서구 마곡동 795-2,강서구,마곡동,5
881102,서울특별시 강서구 마곡동,795-5,795,5,마곡사이언스파크뷰,전세,30.56,20000,2,11,2016,마곡중앙6로 85,오피스텔,18000~20000,마곡사이언스파크뷰,기타,일반,대,185517.525000,450897.540000,2021-06-29,5658,서울특별시 강서구 마곡동 795-5,강서구,마곡동,5
1062895,서울특별시 강서구 화곡동,982-17,982,17,크라운뷰,전세,35.54,2000,50,5,2009,화곡로 274,오피스텔,1억4천 이하,크라운뷰,기타,일반,대,186457.158547,449646.201516,2021-09-08,5729,서울특별시 강서구 화곡동 982-17,강서구,화곡동,12


In [21]:
inv_df['거래유형'].value_counts()

전세    17
Name: 거래유형, dtype: int64

In [22]:
main_data_df.loc[inv_df.index, '거래유형'] = '월세'

In [23]:
negative_df = main_data_df[
    (main_data_df['거래/보증금(만원)'] < 0)
    |(main_data_df['월세(만원)'] < 0)
]

In [24]:
print(negative_df.shape)
negative_df.head()

(0, 26)


,시군구,지번,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,지번주소,구,동,계약날짜기준_건물연식


In [25]:
np.pi

3.141592653589793

In [26]:
np.cos

<ufunc 'cos'>

In [27]:
def year_circular_sine_func(x):
    return np.sin(2*np.pi*((x)/365))

def year_circular_cosine_func(x):
    return np.cos(2*np.pi*((x)/365))

In [28]:
main_data_df['계약날짜일수_sin'] = main_data_df['계약날짜일수_2006년기준'].apply(year_circular_sine_func)
main_data_df['계약날짜일수_cos'] = main_data_df['계약날짜일수_2006년기준'].apply(year_circular_cosine_func)

In [29]:
main_data_df.head()

,시군구,지번,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,지번주소,구,동,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos
0,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,2253,12,12,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-02-22,4070,서울특별시 강남구 개포동 12,강남구,개포동,26,0.811539,0.584298
1,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,730,10,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-04-26,4133,서울특별시 강남구 개포동 12,강남구,개포동,26,0.895839,-0.444378
2,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,1815,4,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-08-31,4260,서울특별시 강남구 개포동 12,강남구,개포동,26,-0.880012,-0.474951
3,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,3865,17,11,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2018-03-29,4470,서울특별시 강남구 개포동 12,강남구,개포동,27,0.999769,0.021516
4,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,3062,7,9,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2018-07-14,4577,서울특별시 강남구 개포동 12,강남구,개포동,27,-0.247022,-0.969010


In [30]:
official_price_df.head()

,구,동,공시지가(원/㎡),기준년도,기준년월,본번,부번,대장구분명
0,강남구,개포동,170000,1990,1990-01-01,1,1,산
1,강남구,개포동,480000,1991,1991-01-01,1,1,산
2,강남구,개포동,1300000,1992,1992-01-01,1,1,산
3,강남구,개포동,1500000,1993,1993-01-01,1,1,산
4,강남구,개포동,1500000,1994,1994-01-01,1,1,산


In [31]:
main_data_df['기준년도'] = main_data_df['계약날짜'].dt.year

In [32]:
main_data_df['본번'].dtype

dtype('int64')

In [33]:
main_data_df['부번'].dtype

dtype('int64')

In [34]:
print(official_price_df.shape)
official_price_df.head()

(30239919, 8)


,구,동,공시지가(원/㎡),기준년도,기준년월,본번,부번,대장구분명
0,강남구,개포동,170000,1990,1990-01-01,1,1,산
1,강남구,개포동,480000,1991,1991-01-01,1,1,산
2,강남구,개포동,1300000,1992,1992-01-01,1,1,산
3,강남구,개포동,1500000,1993,1993-01-01,1,1,산
4,강남구,개포동,1500000,1994,1994-01-01,1,1,산


In [35]:
official_price_df['본번'] = official_price_df['본번'].astype('Int64')
official_price_df['부번'] = official_price_df['부번'].astype('Int64')

In [36]:
main_data_df['본번'] = main_data_df['본번'].astype('Int64')
main_data_df['부번'] = main_data_df['부번'].astype('Int64')

In [37]:
main_data_df.head()

,시군구,지번,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,지번주소,구,동,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,기준년도
0,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,2253,12,12,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-02-22,4070,서울특별시 강남구 개포동 12,강남구,개포동,26,0.811539,0.584298,2017
1,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,730,10,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-04-26,4133,서울특별시 강남구 개포동 12,강남구,개포동,26,0.895839,-0.444378,2017
2,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,1815,4,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-08-31,4260,서울특별시 강남구 개포동 12,강남구,개포동,26,-0.880012,-0.474951,2017
3,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,3865,17,11,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2018-03-29,4470,서울특별시 강남구 개포동 12,강남구,개포동,27,0.999769,0.021516,2018
4,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,3062,7,9,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2018-07-14,4577,서울특별시 강남구 개포동 12,강남구,개포동,27,-0.247022,-0.969010,2018


In [38]:
main_data_df['대장구분명'].value_counts()

일반    5065880
산        2289
Name: 대장구분명, dtype: int64

In [39]:
official_price_df['대장구분명'].value_counts()

일반    29726803
산       513116
Name: 대장구분명, dtype: int64

In [40]:
main_data_df['기준년도'].value_counts()

2020    505195
2019    451913
2015    438439
2016    435564
2017    433355
2018    431650
2021    394716
2014    382295
2013    335664
2011    313830
2012    302108
2006    189431
2007    128238
2009    123099
2008    120481
2010     82191
Name: 기준년도, dtype: int64

In [41]:
official_price_df['기준년도'].value_counts()

1997    1008298
1998    1005989
1999    1000297
1991     996797
1996     993370
2000     991464
1992     986973
2001     984475
1993     977790
2002     974952
1994     974286
1995     966702
2010     959951
2009     959226
2011     957428
2012     955127
2003     948949
2013     948143
2014     940824
2004     939470
2015     934453
2016     933472
2007     930271
2008     927998
2017     924646
2005     921058
2018     920432
1990     919969
2019     914596
2021     906782
2020     906224
2006     629507
Name: 기준년도, dtype: int64

In [42]:
print(main_data_df.shape)

(5068169, 29)


In [43]:
official_price_df['기준년월'] = pd.to_datetime(official_price_df['기준년월'])

In [44]:
official_price_df.head()

,구,동,공시지가(원/㎡),기준년도,기준년월,본번,부번,대장구분명
0,강남구,개포동,170000,1990,1990-01-01,1,1,산
1,강남구,개포동,480000,1991,1991-01-01,1,1,산
2,강남구,개포동,1300000,1992,1992-01-01,1,1,산
3,강남구,개포동,1500000,1993,1993-01-01,1,1,산
4,강남구,개포동,1500000,1994,1994-01-01,1,1,산


In [45]:
official_price_df['기준월'] = official_price_df['기준년월'].dt.month

In [46]:
official_price_df = official_price_df.drop(columns=['기준년월'])
print(official_price_df.shape)
official_price_df.head()

(30239919, 8)


,구,동,공시지가(원/㎡),기준년도,본번,부번,대장구분명,기준월
0,강남구,개포동,170000,1990,1,1,산,1
1,강남구,개포동,480000,1991,1,1,산,1
2,강남구,개포동,1300000,1992,1,1,산,1
3,강남구,개포동,1500000,1993,1,1,산,1
4,강남구,개포동,1500000,1994,1,1,산,1


In [47]:
merge_df = main_data_df.merge(official_price_df, on=['구', '동', '대장구분명', '본번', '부번', '기준년도'])
print(merge_df.shape)
merge_df.head()

(4902600, 31)


,시군구,지번,본번,부번,건물/단지명,거래유형,전용면적(㎡),거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표,계약날짜,계약날짜일수_2006년기준,지번주소,구,동,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,기준년도,공시지가(원/㎡),기준월
0,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,2253,12,12,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-02-22,4070,서울특별시 강남구 개포동 12,강남구,개포동,26,0.811539,0.584298,2017,8930000,1
1,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,730,10,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-04-26,4133,서울특별시 강남구 개포동 12,강남구,개포동,26,0.895839,-0.444378,2017,8930000,1
2,서울특별시 강남구 개포동,12,12,0,SH대치1단지아파트,월세,25.35,1815,4,8,1991,개포로109길 5,아파트,1억4천 이하,SH대치단지,기타,일반,대,206607.181143,443586.061143,2017-08-31,4260,서울특별시 강남구 개포동 12,강남구,개포동,26,-0.880012,-0.474951,2017,8930000,1
3,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,전세,33.18,25000,0,10,1992,개포로109길 9,아파트,24000~26000,성원대치단지,기타,일반,대,206607.181143,443586.061143,2017-01-02,4019,서울특별시 강남구 개포동 12,강남구,개포동,25,0.068802,0.997630,2017,8930000,1
4,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,전세,39.53,33000,0,15,1992,개포로109길 9,아파트,32000~34000,성원대치단지,기타,일반,대,206607.181143,443586.061143,2017-01-03,4020,서울특별시 강남구 개포동 12,강남구,개포동,25,0.085965,0.996298,2017,8930000,1


In [48]:
merge_df = merge_df.rename(columns={'기준월':'공시지가_기준월'})

In [49]:
%%time
merge_df.to_csv('./Prepped Data/main_data_official_land_price_20220124.csv', index=False)

Wall time: 57.2 s
